In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [18]:
# funcion para limpiar los textos de las imagenes
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
# importamos modelo
model_ner = spacy.load('./output/model-best/')

In [24]:
image = cv2.imread('./data/6.jpg') # obtenemos imagen

tessData = pytesseract.image_to_data(image) # exportamos la data

# convertimos la data en dataframe
tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText) # limpiamos los textos

# borramos los datos vacios
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']]) # agregamos un espacio entre dato y dato
print(content)

# obtenemos la prediccion con el modelo
doc = model_ner(content)

cell 8099948528 te 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrikantht@gmail.com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo


In [25]:
from spacy import displacy

In [26]:
# Lo utilizamos para ver los resultados levantando un server
displacy.serve(doc,style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [28]:
# vemos los resultados sin necesidad de levantar un server
displacy.render(doc,style='ent')

### Tagging



- text: es el texto
- ents: son las predicciones
- tokens: son los cortes del texto

In [29]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [30]:
doc_text = docjson['text']

In [31]:
# traemos las palabras en base al token (start & end)

datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,te
3,3,19,29,8466045457
4,4,30,35,email
5,5,36,57,lictsrikant@gmail.com
6,6,58,62,life
7,7,63,72,insurance
8,8,73,84,corporation
9,9,85,87,of


In [32]:
# mergeamos las entidades con los tokens
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [33]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,te,O
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O
5,5,36,57,lictsrikant@gmail.com,B-EMAIL
6,6,58,62,life,B-ORG
7,7,63,72,insurance,I-ORG
8,8,73,84,corporation,I-ORG
9,9,85,87,of,I-ORG


In [34]:
# join lable to df_clean dataframe
# cumsum -> suma acumulada
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

In [35]:
# inner join with start 
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']], how='inner', on='start')

In [36]:
dataframe_info.tail(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
39,5,1,6,3,1,6,495,474,53,21,95,018.,305,301,018,O
40,5,1,6,3,2,1,46,506,430,27,86,"lictsrikant8099948528.blogspot.in,",340,306,lictsrikant8099948528.blogspot.in,O
41,5,1,6,3,2,2,488,506,429,27,86,interviewsinhyderabad.blogspot.in,374,341,interviewsinhyderabad.blogspot.in,B-WEB
42,5,1,6,3,3,1,44,539,472,25,90,"facebook.com/lictsrikant8099948528,",410,375,facebook.com/lictsrikant8099948528,O
43,5,1,6,3,3,2,526,539,443,22,90,facebook.com/thathineni.srikanth.9,445,411,facebook.com/thathineni.srikanth.9,O
44,5,1,6,3,4,1,46,571,106,21,96,promote,453,446,promote,O
45,5,1,6,3,4,2,161,576,56,22,96,your,458,454,your,O
46,5,1,6,3,4,3,226,571,111,21,96,business,467,459,business,O
47,5,1,6,3,4,4,347,571,74,21,92,online,474,468,online,O
48,5,1,6,3,4,5,432,571,96,27,92,pybo,479,475,pybo,O


### Bounding Box

In [53]:
bb_df = dataframe_info.query("label != 'O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to tar

In [38]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,PHONE
3,5,1,3,2,1,2,822,71,197,67,94,8466045457,29,19,8466045457,PHONE
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,ORG
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,ORG


In [39]:
# agrupo por label y orden (fusiono los B con I), al igual que el texto
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [54]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,B-PHONE,21
3,5,1,3,2,1,2,822,71,197,67,94,8466045457,29,19,8466045457,B-PHONE,21
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,B-EMAIL,22
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,B-ORG,23
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,I-ORG,24
8,5,1,3,3,1,3,183,170,117,14,96,corporation,84,73,corporation,I-ORG,24
9,5,1,3,3,1,4,306,170,20,14,96,of,87,85,of,I-ORG,24
10,5,1,3,3,1,5,332,170,42,14,96,india,93,88,india,I-ORG,24
11,5,1,3,3,1,6,668,163,357,46,91,seosrikantht@gmail.com,116,94,seosrikantht@gmail.com,B-EMAIL,25
12,5,1,4,1,1,1,310,228,232,30,92,thathineni,127,117,thathineni,B-NAME,26


In [55]:
# right and bottom of bounding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group,right,bottom
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,B-PHONE,21,1025,81
3,5,1,3,2,1,2,822,71,197,67,94,8466045457,29,19,8466045457,B-PHONE,21,1019,138
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,B-EMAIL,22,1025,167
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,B-ORG,23,79,184
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,I-ORG,24,176,193
8,5,1,3,3,1,3,183,170,117,14,96,corporation,84,73,corporation,I-ORG,24,300,184
9,5,1,3,3,1,4,306,170,20,14,96,of,87,85,of,I-ORG,24,326,184
10,5,1,3,3,1,5,332,170,42,14,96,india,93,88,india,I-ORG,24,374,184
11,5,1,3,3,1,6,668,163,357,46,91,seosrikantht@gmail.com,116,94,seosrikantht@gmail.com,B-EMAIL,25,1025,209
12,5,1,4,1,1,1,310,228,232,30,92,thathineni,127,117,thathineni,B-NAME,26,542,258


In [51]:
# tagging: groupby group
col_group = ['left','top','right','bottom','label','token','group']

# hace una agrupacion por grupos
group_tag_img = bb_df[col_group].groupby(by='group')

In [43]:
# del texto de cada grupo obtengo los datos especificados 
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

In [44]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,822,1025,53,138,[PHONE],8099948528 8466045457
2,709,1025,136,167,[EMAIL],lictsrikant@gmail.com
3,46,374,151,193,[ORG],life insurance corporation of india
4,668,1025,163,209,[EMAIL],seosrikantht@gmail.com
5,310,755,227,259,[NAME],thathineni srikanth
6,399,669,271,296,[DES],insurance advisor
7,47,882,395,427,[ORG],life insurance corporation of india
8,488,917,506,533,[WEB],interviewsinhyderabad.blogspot.in


In [56]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    
    cv2.putText(img_bb,str(label),(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Bounding Box BusinessCard',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to target thread (0x562d4418a2b0)

QObject::moveToThread: Current thread (0x562d4418a2b0) is not the object's thread (0x562d4409e0f0).
Cannot move to tar

# Parser

In [58]:
def parser(text,label):
    if label == 'PHONE':
        text = text.lower()
        text = re.sub(r'\D','',text)
        
    elif label == 'EMAIL':
        text = text.lower()
        allow_special_char = '@_.\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
        
    elif label == 'WEB':
        text = text.lower()
        allow_special_char = ':/.%#\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
        
    elif label in ('NAME', 'DES'):
        text = text.lower()
        text = re.sub(r'[^a-z ]','',text)
        text = text.title()
        
    elif label == 'ORG':
        text = text.lower()
        text = re.sub(r'[^a-z0-9 ]','',text)
        text = text.title()
        
    return text

# Entities

In [59]:
info_array = dataframe_info[['token','label']].values
entities = dict(NAME=[],ORG=[],DES=[],PHONE=[],EMAIL=[],WEB=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]
    
    # step -1 parse the token
    text = parser(token,label_tag)
    
    if bio_tag in ('B','I'):
        
        if previous != label_tag:
            entities[label_tag].append(text)
            
        else:
            if bio_tag == "B":
                entities[label_tag].append(text)
                
            else:
                if label_tag in ("NAME",'ORG','DES'):
                    # -1 es el ultimo
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                    
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
                    
    
    
    previous = label_tag

'Thathineni Srikanth'